In [292]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
import plotly.graph_objs as go
import plotly.offline as py
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
import math


from sklearn import ensemble, linear_model, clone
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import ExtraTreesRegressor, BaggingRegressor


In [256]:
train = pd.read_csv("C:/Users/L3IN/Downloads/enigma18/train.csv")
test = pd.read_csv("C:/Users/L3IN/Downloads/enigma18/test.csv")

In [257]:
train.describe()

,ID,Reputation,Answers,Username,Views,Upvotes
count,330045.000000,3.300450e+05,330045.000000,330045.000000,3.300450e+05,330045.000000
mean,235748.682789,7.773147e+03,3.917672,81442.888803,2.964507e+04,337.505358
std,136039.418471,2.706141e+04,3.579515,49215.100730,8.095646e+04,3592.441135
min,1.000000,0.000000e+00,0.000000,0.000000,9.000000e+00,0.000000
25%,117909.000000,2.820000e+02,2.000000,39808.000000,2.594000e+03,8.000000
50%,235699.000000,1.236000e+03,3.000000,79010.000000,8.954000e+03,28.000000
75%,353620.000000,5.118000e+03,5.000000,122559.000000,2.687000e+04,107.000000
max,471493.000000,1.042428e+06,76.000000,175738.000000,5.231058e+06,615278.000000


In [102]:
train['Reputation'].idxmax()

160728

In [103]:
train[160728:160729]

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
160728,199859,c,1042428.0,4.0,46604,14307.0,11844.0


In [83]:
train['Views'].idxmax()

98585

In [84]:
train[98585:98586]

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
98585,231439,j,15272.0,32.0,96201,5231058.0,144203.0


In [85]:
bin_repo = [-1,1000,10000,50000,100000,500000,1042430]
bin_views = [0,1000,10000,50000,100000,500000,1000000,5231060]

In [86]:
train = train.dropna()

In [87]:
#train['Reputation'] = pd.cut(train['Reputation'].astype(int),bin_repo)
#train['Views'] = pd.cut(train['Views'].astype(int),bin_views)

In [258]:
test['new'] = test['Views']*test['Reputation']
train['new'] = train['Views']*train['Reputation']


In [259]:
train.loc[train['Username'] == 60 ,:]

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,new
84516,81151,s,0.0,2.0,60,45857.0,0.0,0.0


In [260]:
dataset = pd.concat(objs=[train, test], axis=0)
train_l = len(train)

In [261]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
dataset['Tag'] = lb_make.fit_transform(dataset['Tag'])
dataset['Username'] = lb_make.fit_transform(dataset['Username'])
#train['Reputation'] = lb_make.fit_transform(train['Reputation'])
#train['Views'] = lb_make.fit_transform(train['Views'])

In [262]:
train = dataset[:train_l]
test = dataset[train_l:]

In [263]:
test.describe()

,Answers,ID,Reputation,Tag,Upvotes,Username,Views,new
count,141448.000000,141448.000000,1.414480e+05,141448.000000,0.0,141448.000000,1.414480e+05,1.414480e+05
mean,3.914873,235743.073497,7.920927e+03,3.552542,NaN,81348.231117,2.984633e+04,3.220553e+08
std,3.577460,136269.867118,2.791072e+04,2.494553,NaN,49046.098215,8.034374e+04,3.507001e+09
min,0.000000,7.000000,0.000000e+00,0.000000,NaN,4.000000,9.000000e+00,0.000000e+00
25%,2.000000,117797.000000,2.860000e+02,1.000000,NaN,40222.750000,2.608000e+03,1.188850e+06
50%,3.000000,235830.000000,1.245000e+03,4.000000,NaN,78795.500000,8.977000e+03,1.117894e+07
75%,5.000000,353616.000000,5.123000e+03,6.000000,NaN,122149.000000,2.698925e+04,7.483872e+07
max,73.000000,471488.000000,1.042428e+06,9.000000,NaN,175737.000000,5.004669e+06,4.656757e+11


In [264]:
avr = train['Reputation'].mean()
avv = train['Views'].mean()
sr = train['Reputation'].std()
sv = train['Views'].std()
nr = train['new'].mean()
nv = train['new'].std()

In [137]:

#train['Answers'] = train['Answers']/10

In [116]:
d = {}
max_user = train['Username'].max()
val = train.shape[0]

for i in range(max_user+1):
    d[i] = 0

for i in range(val):
    d[int(train.ix[i]['Username'])] += 1

In [139]:
train['uv'] = train['Upvotes']/train['Views']

In [140]:
v = {}
max_user = train['Username'].max()
val = train.shape[0]

for i in range(max_user+1):
    v[i] = 0

for i in range(val):
    temp = train.ix[i]
    uv = temp['uv']
    x1 = int(temp['Username'])
    v[x1] += uv/d[x1]


In [265]:
train['Reputation'] = (train['Reputation'] - train['Reputation'].mean()) / (train['Reputation']).std()
train['Views'] = (train['Views'] - train['Views'].mean()) / (train['Views']).std()
train['new'] = (train['new'] - train['new'].mean()) / (train['new']).std()

In [175]:
v

{0: 0.00026673068203035398,
 1: 0.0,
 2: 3.6828343092844256e-05,
 3: 0.003111148994203887,
 4: 0,
 5: 0.0048848569434752267,
 6: 0.0065292096219931274,
 7: 0,
 8: 0,
 9: 0.005763688760806916,
 10: 3.0344869441199228e-05,
 11: 2.4897918534010556e-05,
 12: 0.0078131807487873597,
 13: 0.021459227467811159,
 14: 0.0010387451958034693,
 15: 0,
 16: 0,
 17: 0.011097410604192354,
 18: 0.010994502748625687,
 19: 0,
 20: 0,
 21: 0.0,
 22: 0.021897810218978103,
 23: 0,
 24: 0,
 25: 0,
 26: 0.0013404825737265416,
 27: 0,
 28: 0.0035325287017957019,
 29: 0.00091095422455021637,
 30: 0.0023617153511497823,
 31: 0.0007326007326007326,
 32: 0.00056122448979591837,
 33: 0,
 34: 0,
 35: 0,
 36: 0.0010604453870625664,
 37: 0.015902712815715623,
 38: 2.264077467674634e-06,
 39: 8.0019204609106182e-05,
 40: 0,
 41: 6.5420764550668388e-05,
 42: 0,
 43: 0.000161138095347713,
 44: 0,
 45: 0,
 46: 0.0076577571607392685,
 47: 0.0005294927459493805,
 48: 0,
 49: 0.0037917959324370908,
 50: 0.0027386541471048514

In [176]:
train['Username'] = train['Username'].map(v)

In [187]:
test

,Answers,ID,Reputation,Tag,Upvotes,Username,Views
0,3.0,366953,-0.078641,0,NaN,0.006919,0.043912
1,6.0,71864,0.618514,1,NaN,0.029499,-0.332464
2,1.0,141692,-0.252986,3,NaN,0.002964,-0.104724
3,6.0,316833,-0.286465,3,NaN,0.000216,-0.137322
4,10.0,440445,-0.121876,3,NaN,0.017881,0.340861
5,1.0,3514,-0.167070,1,NaN,0.006969,-0.337689
6,1.0,331420,-0.255498,9,NaN,0.020569,-0.285809
7,4.0,406540,-0.258787,1,NaN,0.000424,0.347470
8,2.0,227379,0.035248,8,NaN,0.007295,-0.311106
9,4.0,331687,-0.280959,6,NaN,0.011371,-0.305671


In [275]:
X = train[['Tag','Views','new','Reputation']]
y = train[['Upvotes']]

In [276]:
test_v = test['Views']

In [277]:
test['Reputation'] = (test['Reputation'] - avr) / sr
test['Views'] = (test['Views'] - avv) / sv
test['new'] = (test['new'] - nr) / nv

In [ ]:
test['Username'] = test['Username'].map(v)

In [186]:
test = test.replace({'Username': {0.0: test['Username'].mean()}})

In [278]:
x_test = test[['Tag','Views','new','Reputation']]

In [279]:
val_size = 0.1
seed = 0

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=seed)

In [280]:
scorer = make_scorer(mean_squared_error, greater_is_better = False)

In [210]:
ensemble = ensemble.GradientBoostingRegressor(n_estimators = 75,
                                              max_depth = 5, 
                                              min_samples_split = 2)

gbr = ensemble.fit(x_train, y_train)
print(ensemble.score(x_val, y_val))
#print(np.sqrt(-cross_val_score(gbr, x_train, y_train, scoring = scorer, cv = 5))

AttributeError: 'GradientBoostingRegressor' object has no attribute 'GradientBoostingRegressor'

In [195]:
ols_simple = linear_model.LinearRegression()

lr = ols_simple.fit(x_train, y_train)
print(ols_simple.score(x_val, y_val))
print(np.sqrt(-cross_val_score(lr, x_train, y_train, scoring = scorer, cv = 5)))

0.279602740204
[ 2223.07112334  3522.35419409  3075.93546284  4190.4107827   2954.84706548]


In [306]:
ridgecv = linear_model.RidgeCV(alphas = [0.001, 0.01, 0.1, 0.5, 0.75, 1, 1.2, 1.5, 2.5, 5])

ridge = ridgecv.fit(x_train, y_train)
print(ridgecv.score(x_val, y_val))
print(np.sqrt(-cross_val_score(ridge, x_train, y_train, scoring = scorer, cv = 5)))

0.934377121614
[  863.16333954  1281.75743713   986.74164584  1638.30873999   959.31992945]


In [247]:
model_xgb = xgb.XGBRegressor(
                             learning_rate=0.05, max_depth=4, 
                            n_estimators=150, seed = 1894,
                              nthread = -1)

xg = model_xgb.fit(x_train, y_train)
print(model_xgb.score(x_val, y_val))
#print(np.sqrt(-cross_val_score(xg, x_train, y_train, scoring = scorer, cv = 5)).mean())

0.911476821667


In [242]:
from sklearn.preprocessing import RobustScaler
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

lass = lasso.fit(x_train, y_train)
print(lasso.score(x_val, y_val))
print(np.sqrt(-cross_val_score(lass, x_train, y_train, scoring = scorer, cv = 5)))

0.304120664324
[ 2132.01557516  3438.04028563  2997.03022946  4134.60272928  2891.83498791]


In [125]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

elas = ENet.fit(x_train, y_train)
print(elas.score(x_val, y_val))
print(np.sqrt(-cross_val_score(elas, x_train, y_train, scoring = scorer, cv = 5)))

0.098273073253
[ 2457.38492847  3838.61704482  3287.26560416  4593.70392905  3214.23088685]


In [281]:
rf = RandomForestRegressor(n_estimators=50, random_state=47, n_jobs=-1)

rand = rf.fit(x_train,y_train)
print(rf.score(x_val,y_val))
#print(np.sqrt(-cross_val_score(rand, x_train, y_train, scoring = scorer, cv = 5)).mean())

0.918982713365


In [224]:
xt = ExtraTreesRegressor(n_estimators=100)

xt = xt.fit(x_train,y_train)
print(xt.score(x_val,y_val))

0.895333403012


In [307]:
y_pred = ridge.predict(x_test)

In [297]:
bg = BaggingRegressor(ridge, n_estimators=10, n_jobs=-1)
bg.fit(x_train,y_train)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.

In [298]:
mean_20 = train.loc[train['Upvotes'] <= 20, 'Upvotes'].mean()

In [299]:
mean_20

7.846120204923154

In [302]:
ans = []

for i in y_pred:
    if i[0] < 0:
        i[0] = mean_20
    ans.append(i[0])
               
ans = np.array(ans)

In [303]:
ans

array([ 226.94482383,   92.61290065,   37.59997065, ...,   87.27893992,
          0.98643793,   19.31768499])

In [305]:
sub = pd.DataFrame({
    'ID' : test['ID'],
    'Upvotes' : ans
})
sub.to_csv('C:/Users/L3IN/Downloads/enigma18/sub14.csv', index=False)

In [304]:
np.sum(ans < 0)

0